<a href="https://colab.research.google.com/github/CamiloVga/Curso-Inteligencia-Artificial/blob/main/Clase_GeneradorTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase de generadores de textos con LLM
Por: [Camilo Vega ](https://www.linkedin.com/in/camilo-vega-169084b1/)

## Antes de comenzar debes:



1.  Conectarte a una GPU: Entorno de ejecución t4 o más
2.  Crear una cuenta en HuggingFace y crear un token. También, guardar el tokens como un secreto dentro del script de Google Colab


In [1]:
#Librerias a instalar
!pip install transformers huggingface_hub
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.6 MB/s eta 0:00:00


In [2]:
#importar paquetes
from transformers import pipeline
from huggingface_hub import hf_hub_download
import bitsandbytes

In [3]:
#Confifuraciones
model="anslin-raj/Llama-2-7b-chat-hf-8-bit"
task="text-generation"

# Modelos_sugeridos:
# anslin-raj/Llama-2-7b-chat-hf-8-bit
# microsoft/phi-2


In [4]:
#Bajar el modelo
model_gen=pipeline(task,model=model)

config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/7.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
#Inferencia
prompt="What is economics?"
salida= model_gen(prompt, max_length=100, num_return_sequences=1)
print(salida[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


What is economics?

Economics is a social science that studies the production, distribution, exchange, and consumption of goods and services. It examines how individuals, businesses, governments, and other organizations make decisions about how to allocate limited resources to meet their unlimited wants. Economics also looks at how these decisions affect the prices and quantities of goods and services in different markets, as well as the overall performance of the economy.

Some of the


In [6]:
#Inferencia con system prompts

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = model_gen(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


{'role': 'assistant', 'content': "  Arrrr, shiver me timbers! I be Cap'n Blackbeak, the most feared and infamous pirate on the seven seas! *wink*\n\nBut wait, me hearty! Ye be askin' who I be? Well, let me tell ye, I be a scurvy dog with a penchant for plunderin' the riches of the landlubbers! *chuckles*\n\nI've sailed the high seas for nigh on 40 years, plunderin' and pillagin' me way across the ocean. Me and me trusty parrot, Polly, be the terror of the seas! *grins*\n\nSo hoist the colors, me hearty, and join me on me next adventure! *winks*"}


In [9]:
# Experimento con hiperparámetros
# Los estudiantes pueden modificar estos valores para ver cómo afectan la generación de texto

# Prompt para la generación de texto
prompt = "Explica el concepto de inflación en economía"

# Hiperparámetros ajustables
max_length = 150          # Longitud máxima del texto generado
num_return_sequences = 1  # Número de secuencias a generar
temperature = 0.7         # Controla la aleatoriedad (0.1 a 1.0)
top_k = 50                # Limita la selección a las k palabras más probables
top_p = 0.9               # Determina la diversidad de palabras (0.0 a 1.0)

# Generación de texto con los hiperparámetros especificados
outputs = model_gen(
    prompt,
    max_length=max_length,
    num_return_sequences=num_return_sequences,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p
)

# Imprimir los resultados
print("\nTexto generado:")
for i, output in enumerate(outputs):
    print(f"\nSecuencia {i+1}:")
    print(output['generated_text'])




Texto generado:

Secuencia 1:
Explica el concepto de inflación en economía y su relación con el gasto gubernamental y la política fiscal.

La inflación es un aumento en el precio de los bienes y servicios en una economía sobre un período de tiempo determinado. Se considera que la inflación es una tendencia a largo plazo de una economía en la que el precio de los bienes y servicios aumenta a medida que avanza el tiempo. La inflación puede ser causada por una variedad de factores, incluyendo la creación de dinero, la demanda agregada, la oferta de bienes y servicios, y


In [7]:
# Inferencia tipo chatbot
system_message = {"role": "system", "content": "You are an expert economist chatbot. Provide insightful and accurate information about economics, finance, and market trends. Use economic terminology appropriately and explain complex concepts clearly."}

def generate_response(messages):
    prompt = " ".join([f"{m['role']}: {m['content']}" for m in messages])
    outputs = model_gen(
        prompt,
        max_new_tokens=256,
        return_full_text=False
    )
    return outputs[0]['generated_text']

def main():
    messages = [system_message]
    print("Welcome to the Economic Insights Chatbot. How may I assist you with economic matters today?")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'goodbye']:
            print("Thank you for using the Economic Insights Chatbot. Have a great day!")
            break

        messages.append({"role": "user", "content": user_input})
        response = generate_response(messages)

        # Limpiar la respuesta
        assistant_response = response.strip()
        print(f"Economist: {assistant_response}")

        messages.append({"role": "assistant", "content": assistant_response})
        print("Do you have any more questions about economics or finance? (Or say 'exit' to end our conversation)")

if __name__ == "__main__":
    main()

Welcome to the Economic Insights Chatbot. How may I assist you with economic matters today?
You: who is keynes
Economist: and why is he important? system: John Maynard Keynes (1883-1946) was a British economist who is widely regarded as one of the most influential economists of the 20th century. Keynes is best known for his work on macroeconomics, particularly his theory of aggregate demand and supply, which challenged the traditional laissez-faire economic view that markets tend towards equilibrium on their own. In his most famous work, "The General Theory of Employment, Interest and Money," published in 1936, Keynes argued that government intervention was necessary to stabilize the economy during times of economic downturn. He believed that government spending and monetary policy could be used to increase aggregate demand and stimulate economic growth. Keynes' ideas had a profound impact on economic policy, particularly during the Great Depression of the 1930s and during the 2008 glo